## Model (Count Vectorize + Bag Of Word + Tfidf Transform + SGD classification)

### Bài toán
- Phân loại văn bản theo 4 loại cảm xúc cơ bản (neutral, happy, angry, sad).

### Mục tiêu
- Tìm hiểu được cách crawl văn bản từ Google API.
- Tìm hiểu được cách preprocessing văn bản.
- Tìm hiểu được các cách biến đổi từ văn bản thành vector.
- Tìm hiểu được và xây dựng một nền tảng để có thể phân loại được cảm xúc khi nhập vào một đoạn văn bản.

### Ứng dụng
- Có thể ứng dụng vào phân loại các cảm xúc của các comment từ đó rút ra thống kê, có thể ứng dụng vào trong các ứng dụng như: phân loại comment đánh giá sản phẩm (e-commerce), phân loại comment đánh giá trên facebook post, xếp loại video youtube bằng cách phân loại comment.
- Ứng dụng để gợi ý emoji trong các loại ứng dụng chat.

### Crawl dữ liệu
#### API
- Sử dụng GoogleAPIs crawl comment từ các video youtube, mỗi url video đa số chứa những comment thuộc cùng một nhóm. Ví dụ: video về nền công nghiệp gà, bò, hay chiến tranh thì đa số là angry, cũng có sad, các video âm nhạc giải trí mang tính thư giãn thì happy, những video của các vlogger thì đa phần là hỗn tạp vì có người thích và có người ghét.
    1. WjNFGZLJLss (angry)
    2. 5xzGjW_MEms (sad)
    3. nKDgFCojiT8 (sad)
    4. hEH7KgQY380 (happy)
    5. kIF3BYBXZWA (happy)
    6. NvZtkt9973A (happy)
    7. SuYwm-wBwBY (happy)
    8. iqmO1RlqorU (happy)
    9. 9ca8ThA83fE (happy)
    10. V-RwqjtQmm8 (angry)
    11. WuLZ9ZMA_FA (angry)
    12. kopI4-ebPxQ (prank)
    13. LEBtUTAf8uE (prank)
    14. eBSr1oiIDuU (prank)
    15. UYJl7z38V88 (prank)

#### Thông tin dữ liệu crawl
- Dữ liệu ở đây lên đến ~35k comment.
- Thông tin dữ liệu crawl chỉ là những comment, reply, không có label.
- Dẫn đến việc nhóm phải tự annotation lại dữ liệu bằng tay.

#### Annotation
- Nhóm có tự tạo file annotation.py để hỗ trợ việc annotate nhanh hơn và lưu lại cache để ghi nhớ những comment chưa annotate.
- Dữ liệu sau khi nhóm tự annotate vào khoảng ~12k comment.
- Các file dữ liệu dùng để anotate bao gồm:
    - encoded_angry.txt
    - encoded_happy.txt
    - encoded_sad.txt
    - encoded_prank.txt
- Nhóm sử dụng file script anotate để tạo thuận tiện cho việc anotate hơn.
- Kết quả sau khi anotate là các file:
    - final_angry.txt
    - final_happy.txt
    - final_sad.txt
    - final_prank.txt


### Preprocessing data
- Do các bình luận có thể là những trường hợp như:
    - Viết tắt: "omg", "asap", "tbh", ...
    - Viết sai chính tả
    - Viết nhảm (spam)
    - Có tên người
    - ...
- Nên nhóm sẽ sử dụng file "glove.6B.50d.txt" để lọc bớt những từ hợp lệ, chừa các tự ngoại lệ ra để nhóm xem xét bỏ hay giữa để sửa lại (*).

- Các bước chuẩn bị dữ liệu:
    - Chỉ giữa các kí tự, xóa các kí tự đặc biệt và số, xóa các dòng text là rỗng
    - Thực hiện (*)
    - Map các từ ngoại lệ bằng các giá trị mà nhóm đã tự nhập, sau đó chuẩn hóa chuỗi lại, bỏ các khoảng trắng thừa, và xóa dòng text rỗng.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
import codecs
import json
%matplotlib inline

In [2]:
# Load các file csv vào thành 1 DataFrame
happy_train = pd.read_csv('./CrawlComment/final_happy.csv', sep='\t', header=None)
angry_train = pd.read_csv('./CrawlComment/final_angry.csv', sep='\t', header=None)
sad_train = pd.read_csv('./CrawlComment/final_sad.csv', sep='\t', header=None)
prank_train = pd.read_csv('./CrawlComment/final_prank.csv', sep='\t', header=None)
frames = [happy_train, angry_train, sad_train, prank_train]
df = pd.concat(frames, ignore_index=True)
df.info()

f = codecs.open("./CrawlComment/glove.6B.50d.txt","r","utf-8")
lines = f.readlines()
words = [x.split(' ')[0] for x in lines]
glove_dict = {}
for word in words:
    glove_dict[word] = word
f.close()
# Mở file đang làm dang dở trước đó (nếu có)
glove_dict.update(json.loads(open(input("Enter custom dictionary file: "), "r").read()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12864 entries, 0 to 12863
Data columns (total 2 columns):
0    12864 non-null object
1    12864 non-null int64
dtypes: int64(1), object(1)
memory usage: 201.1+ KB
Enter custom dictionary file: dictionary.txt


In [3]:
def normalize(x):
    try:
        x = x.lower().strip()
    except:
        return np.nan
    ans = ''
    for i in x:
        if 'a' <= i <= 'z' or i == ' ':
            ans += i
    return ans if len(ans) > 0 else np.nan

def keep_clean(x):
    x = x.strip()
    return ' '.join(x.split())


# Class Custom có nhiệm vụ chỉ giữ kí tự trong chuỗi, nếu len = 0 thì sẽ là giá trị bị thiếu
class Custom(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_new = X
        X_new[0] = X_new[0].apply(lambda x: normalize(x))
        X_new.dropna(inplace=True)
        return X_new

# Class Custom2 có nhiệm vụ bỏ các khoảng trắng thừa trong chuỗi.
class Custom2(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_new = X
        X_new[0] = X_new[0].apply(lambda x: keep_clean(x))
        X_new.dropna(inplace=True)
        return X_new

# Class Interactive, nếu có những từ không có trong từ điển, ta phải nhập bằng tay hahahaha
class Interactive(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        text = X[0]
        new = []
        for line in text:
            new_line = ''
            for word in line.split():
                if word not in glove_dict:
                    print(word)
                    glove_dict[word] = input(word + ' ')
                    open("dictionary.txt", "w").write(json.dumps(glove_dict))
                else:
                    new_line += ' ' + glove_dict[word]
            new.append(new_line)
        X[0] = new
        return X
preprocess = make_pipeline(
    Custom(),
    Interactive(),
    Custom(),
    Custom2()
)

preprocess.fit_transform(df)
df.info()
df.to_csv('final.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12742 entries, 1 to 12863
Data columns (total 2 columns):
0    12742 non-null object
1    12742 non-null int64
dtypes: int64(1), object(1)
memory usage: 298.6+ KB


In [ ]:
### Xây dựng model
1. Đầu tiên, nhóm ghép các csv đã annotate vào một df bằng lệnh `pd.concat(frames)`
2. Sau đó, nhóm split thành 4 tập `X_train, X_test, y_train, y_test`, train và test theo tỉ lệ 7/3, có shuffle.
3. Sử dụng tư tưởng `Bag Of Word` để xây dựng feature cho model, nhóm sử dụng `CountVectorize()` để xây dựng feature đó trên tập X_train. Lưu feature đó vào tập `X_train_counts`.
4. Vì không thể lấy số lượng chữ để train, vì nếu gặp văn bản có nhiều word so với văn bản có ít word sẽ có sự khác nhau rất lớn mặc dù về label sẽ không có sự khác nhau. Nên ta quy feature về tần suất xuất hiện của các word trong văn bản. Nhóm sử dụng `TfidfTransformer()` để transform từ số lượng word xuất hiện qua tần suất. Sau đó lưu vào ma trận tần suất `X_train_tfidf`.
5. Sau đó nhóm sử dụng ma trận tần suất `X_train_tfidf` để feed qua một lớp classifier để phân loại. Ở đây nhóm sử dụng `SGDClassifier` để phân loại.

In [2]:
df = pd.read_csv('./final.csv', sep='\t', header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12864 entries, 0 to 12863
Data columns (total 2 columns):
0    12864 non-null object
1    12864 non-null int64
dtypes: int64(1), object(1)
memory usage: 201.1+ KB


In [3]:
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, shuffle=True)

print(X_train.shape)
print(y_train.shape)
print(y_train.unique())
print(y_train.value_counts())

(9004,)
(9004,)
[0 2 3 1]
3    2649
0    2390
1    2202
2    1763
Name: 1, dtype: int64


In [4]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)
print(count_vect.get_feature_names())

(9004, 8425)
['00', '000', '001', '00212', '00am', '01', '02', '03', '04', '05', '06', '0613361717', '07', '08', '09', '0wppwpwpwpwpwpwp', '0x0', '10', '100', '1000', '100000', '10000000000', '10000dollars', '1000x', '101', '107', '108oneness', '109', '10th', '11', '1111', '115', '116', '11k', '12', '12678773020', '12k', '13', '135', '14', '144', '148', '14hfebto', '15', '150', '15000', '15m', '15mil', '15millon', '15min', '16', '17', '18', '181flo', '18milion', '19', '1941', '1974', '1m', '1otxs2ybyo', '1sec', '1st', '20', '200', '2000', '2002', '2012', '2013', '2015', '2016', '2017', '2018', '2019', '2020', '2020ians', '2021', '2027', '20k', '20s', '20th', '20xx', '20years', '21', '2100', '22', '22y', '23', '24', '24h', '25', '250k', '25a', '26', '263', '27', '28', '29', '2k', '2nd', '2x20', '30', '300', '302_1514860746', '303', '30seconds', '30th', '31', '32', '33', '333', '34', '35', '36', '36am', '37', '378', '38', '384', '39', '3am', '3k', '3rabshort', '3rd', '3this', '40', '40k'

In [5]:
count_vect.vocabulary_.get(u'music')

4969

In [6]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(9004, 8425)

In [7]:
SGD_clf = SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-4, random_state=42,
                          max_iter=100, tol=None)

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGD_clf),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.6471502590673575

In [8]:
print(metrics.classification_report(y_test.astype(str), predicted.astype(str),
    target_names=['0', '1', '2', '3']))

metrics.confusion_matrix(y_test.astype(str), predicted.astype(str))

              precision    recall  f1-score   support

           0       0.62      0.63      0.62      1046
           1       0.69      0.71      0.70       926
           2       0.58      0.52      0.55       739
           3       0.69      0.70      0.69      1149

    accuracy                           0.65      3860
   macro avg       0.64      0.64      0.64      3860
weighted avg       0.65      0.65      0.65      3860



array([[656, 113, 127, 150],
       [105, 655,  63, 103],
       [144,  95, 385, 115],
       [161,  92,  94, 802]], dtype=int64)